## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cape Town,ZA,-33.93,18.42,61.00,87,86,8.05,overcast clouds
1,Bredasdorp,ZA,-34.53,20.04,57.20,93,93,3.36,overcast clouds
2,Rikitea,PF,-23.12,-134.97,74.62,81,100,20.31,light rain
3,Mengcheng,CN,33.26,116.57,69.53,32,71,2.24,broken clouds
4,Kahului,US,20.89,-156.47,84.20,62,1,5.82,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Max Temp"] <= max_temp)]
clean_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,Kahului,US,20.89,-156.47,84.20,62,1,5.82,clear sky
5,Port Blair,IN,11.67,92.75,82.76,81,99,18.86,moderate rain
6,Biak,ID,-0.91,122.88,83.62,67,100,6.64,light rain
9,Kapaa,US,22.08,-159.32,78.80,83,40,10.29,scattered clouds
16,Lufilufi,WS,-13.87,-171.60,86.00,70,75,3.36,broken clouds
...,...,...,...,...,...,...,...,...,...
664,Damietta,EG,31.42,31.81,77.00,78,20,5.82,few clouds
668,Myitkyina,MM,25.38,97.40,81.10,69,92,4.54,overcast clouds
678,Jeremie,HT,18.65,-74.12,79.11,75,94,3.51,heavy intensity rain
682,Nhamunda,BR,-2.19,-56.71,78.19,62,68,1.90,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City                   183
Country                183
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
dtype: int64

In [6]:
clean_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace=True)
clean_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,Kahului,US,20.89,-156.47,84.20,62,1,5.82,clear sky
5,Port Blair,IN,11.67,92.75,82.76,81,99,18.86,moderate rain
6,Biak,ID,-0.91,122.88,83.62,67,100,6.64,light rain
9,Kapaa,US,22.08,-159.32,78.80,83,40,10.29,scattered clouds
16,Lufilufi,WS,-13.87,-171.60,86.00,70,75,3.36,broken clouds
...,...,...,...,...,...,...,...,...,...
664,Damietta,EG,31.42,31.81,77.00,78,20,5.82,few clouds
668,Myitkyina,MM,25.38,97.40,81.10,69,92,4.54,overcast clouds
678,Jeremie,HT,18.65,-74.12,79.11,75,94,3.51,heavy intensity rain
682,Nhamunda,BR,-2.19,-56.71,78.19,62,68,1.90,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kahului,US,84.20,clear sky,20.89,-156.47,
5,Port Blair,IN,82.76,moderate rain,11.67,92.75,
6,Biak,ID,83.62,light rain,-0.91,122.88,
9,Kapaa,US,78.80,scattered clouds,22.08,-159.32,
16,Lufilufi,WS,86.00,broken clouds,-13.87,-171.60,
21,Cayenne,GF,75.20,clear sky,4.93,-52.33,
22,La Orilla,MX,78.53,overcast clouds,17.98,-102.23,
24,Grand Gaube,MU,79.00,clear sky,-20.01,57.66,
30,Labuhan,ID,80.65,light rain,-6.88,112.21,
32,Thinadhoo,MV,83.01,broken clouds,0.53,72.93,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kahului,US,84.20,clear sky,20.89,-156.47,Maui Seaside Hotel
5,Port Blair,IN,82.76,moderate rain,11.67,92.75,Fortune Resort Bay Island
6,Biak,ID,83.62,light rain,-0.91,122.88,Kost Ariz
9,Kapaa,US,78.80,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
16,Lufilufi,WS,86.00,broken clouds,-13.87,-171.60,Leuaina Beach Resort and Spa
21,Cayenne,GF,75.20,clear sky,4.93,-52.33,Hôtel Le Dronmi
22,La Orilla,MX,78.53,overcast clouds,17.98,-102.23,Baymont by Wyndham Lazaro Cardenas
24,Grand Gaube,MU,79.00,clear sky,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
30,Labuhan,ID,80.65,light rain,-6.88,112.21,bador sepret
34,Lorengau,PG,83.62,moderate rain,-2.02,147.27,Lorengau Harbourside Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index=False)

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))